In [7]:
import os

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

print(tf.version.VERSION)

2.10.0


In [8]:
dataset = pd.read_csv('../datas/RGB_color_labels.csv')
dataset = pd.get_dummies(dataset, columns=['label'])

In [9]:
train_dataset = dataset.sample(frac=0.8, random_state=8) #train_dataset = 80% of total dataset  
#random_state = any int value means every time when you run your program you will get the same output for train and test dataset, random_state is None by default which means every time when you run your program you will get different output because of splitting between train and test varies within 
test_dataset = dataset.drop(train_dataset.index) #remove train_dataset from dataframe to get test_dataset

In [10]:
train_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    train_labels[x] = train_dataset.pop(x)

In [11]:
test_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    test_labels[x] = test_dataset.pop(x)

In [12]:
train_dataset = np.array(train_dataset)
train_labels = np.array(train_labels)
train_labels = np.argmax(train_labels, axis=1)

test_dataset = np.array(test_dataset)
test_labels = np.array(test_labels)
test_labels = np.argmax(test_labels, axis=1)


In [13]:
def create_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(3, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=(3,)))
    for i in range(1,5):
        model.add(tf.keras.layers.Dense(15, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))
    model.add(tf.keras.layers.Dense(11))

    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    model.build()

    return model

In [14]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 12        
                                                                 
 dense_1 (Dense)             (None, 15)                60        
                                                                 
 dense_2 (Dense)             (None, 15)                240       
                                                                 
 dense_3 (Dense)             (None, 15)                240       
                                                                 
 dense_4 (Dense)             (None, 15)                240       
                                                                 
 dense_5 (Dense)             (None, 11)                176       
                                                                 
Total params: 968
Trainable params: 968
Non-trainable pa

In [15]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 verbose=1,
                                                 save_best_only=True,
                                                 save_weights_only=True,
                                                 save_freq=10)

# Train the model with the new callback
model.fit(train_dataset, 
          train_labels,  
          epochs=900,
          batch_size = 1000,
          validation_data=(test_dataset, test_labels),
          callbacks=[cp_callback], # Pass callback to training
          shuffle=True)  

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/900
5/5 [==============================] - 1s 68ms/step - loss: 2.4436 - accuracy: 0.0995 - val_loss: 2.3402 - val_accuracy: 0.1267
Epoch 2/900
5/5 [==============================] - 0s 10ms/step - loss: 2.3190 - accuracy: 0.1692 - val_loss: 2.2290 - val_accuracy: 0.2822
Epoch 3/900
5/5 [==============================] - 0s 8ms/step - loss: 2.2377 - accuracy: 0.2771 - val_loss: 2.1504 - val_accuracy: 0.3000
Epoch 4/900
5/5 [==============================] - 0s 8ms/step - loss: 2.1762 - accuracy: 0.2843 - val_loss: 2.0978 - val_accuracy: 0.3020
Epoch 5/900
5/5 [==============================] - 0s 8ms/step - loss: 2.1320 - accuracy: 0.2850 - val_loss: 2.0585 - val_accuracy: 0.3020
Epoch 6/900
5/5 [==============================] - 0s 8ms/step - loss: 2.0975 - accuracy: 0.2850 - val_loss: 2.0252 - val_accuracy: 0.3020
Epoch 7/900
5/5 [==============================] - 0s 8ms/step - loss: 2.0684 - accuracy: 0.2850 - val_loss: 1.9976 - val_accuracy: 0.3020
Epoch 8/900
5/5 [========

In [16]:
model.evaluate(test_dataset, test_labels, verbose=2)

32/32 - 0s - loss: 0.5833 - accuracy: 0.7624 - 93ms/epoch - 3ms/step


[0.5833496451377869, 0.7623762488365173]

In [17]:
# this saves the whole model including the loss function used, everythin etc.
save_path = "saved_models"
model.save(f"{save_path}/model1")

INFO:tensorflow:Assets written to: saved_models/model1\assets


In [18]:
# this loads the whole model saved
# verbose just means what do you want to see in the outputs(what metrics do you want to see etc.)
new_model = tf.keras.models.load_model(f"{save_path}/model1")
new_model.evaluate(test_dataset, test_labels, verbose=2)

32/32 - 0s - loss: 0.5833 - accuracy: 0.7624 - 216ms/epoch - 7ms/step


[0.5833496451377869, 0.7623762488365173]

In [36]:
# this part saves the weights in a excel file
import xlsxwriter
relative_row_idx = 0 # this is for writing bias
row_idx = 0
max_row_idx = 0
column_idx = 0

workbook = xlsxwriter.Workbook("../datas/saved_weights_bias/1/saved_weights_biases.xlsx")
worksheet = workbook.add_worksheet()

In [37]:
for layer in model.layers:
    for row_weights in layer.get_weights()[0].T:
        for weights in row_weights:
            worksheet.write(row_idx, column_idx, weights)
            column_idx += 1
        
        column_idx += 1
        print(relative_row_idx)
        try:
            worksheet.write(row_idx, column_idx, layer.get_weights()[1].T[relative_row_idx])
        except:
            pass
        relative_row_idx += 1
        row_idx += 1

        column_idx = 0
    relative_row_idx = 0
    row_idx += 2


workbook.close()

0
1
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
1
2
3
4
5
6
7
8
9
10
